# VAMP

For users already familiar with the VAMP interface: The corresponding [API docs](../api/generated/sktime.decomposition.VAMP.rst#sktime.decomposition.VAMP).

VAMP is short for [V]ariational [A]pproach for [M]arkov [P]rocesses and was introduced in <cite data-cite="wu2020variational">(Wu, 2020)</cite>. It can be used as linear dimensionality reduction tool and extends [TICA](tica.ipynb) in the sense that it gives rise to a systematic way to select input features as well as handles off-equilibrium data consistently.

While the VAMP model possesses methods to project data into lower dimensions, it also is equipped with scoring functions (the so-called VAMP scores), which allow ranking of features. This has been demonstrated in, e.g., <cite data-cite="scherer2019variational">(Scherer, 2019)</cite>.

In [ ]:
import numpy as np  # NumPy for general numerical operations
import sktime

In short, a VAMP model can be estimated by first creating an estimator

In [ ]:
vamp_estimator = sktime.decomposition.VAMP(lagtime=1, dim=.9, scaling="kinetic_map")

In [ ]:
ellipsoids = sktime.data.ellipsoids(seed=17)  # create dataset instance
discrete_trajectory = ellipsoids.discrete_trajectory(n_steps=1000)  # discrete transitions
feature_trajectory = ellipsoids.map_discrete_to_observations(discrete_trajectory)  # corresponding observations

In [ ]:
vamp = sktime.decomposition.VAMP(
    lagtime=1,  # time shift is one step
    dim=1,  # fix projection dimension explicitly
    scaling=None,
    right=False,
    epsilon=1e-5,
    ncov=5
)

In [ ]:
vamp.fit(feature_trajectory)

In [ ]:
model = vamp.fetch_model()

Model contains covariances model (from covariance estimator)

In [ ]:
model.cov

And also gives direct access to c00, c0t, ctt

In [ ]:
model.cov_00, model.cov_0t, model.cov_tt

Cumulative variance

In [ ]:
model.cumvar

Dimension parameter of estimator

In [ ]:
model.dim

Epsilon parameter of estimator

In [ ]:
model.epsilon

Compute future expectation of observable or covariance using the approximated Koopman operator.

In [ ]:
model.expectation  # currently broken

In [ ]:
model.mean_0, model.mean_t

In [ ]:
model.output_dimension

In [ ]:
model.right

In [ ]:
model.scaling

In [ ]:
model.score(test_model=None, score_method="VAMP2")  # todo: this can be reduced to method="VAMP2"

In [ ]:
model.singular_values

In [ ]:
model.singular_vectors_left, model.singular_vectors_right

In [ ]:
projection = model.transform(feature_trajectory)

In [ ]:
import matplotlib as mpl
import matplotlib.animation as animation
import matplotlib.pyplot as plt
plt.plot(projection)

In [ ]:
from sktime.data.pbf import PBF

In [ ]:
interaction_distance = 1.5
init_pos_x = np.arange(-24, 24, interaction_distance*.9).astype(np.float32)
init_pos_y = np.arange(0, 24, interaction_distance*.9).astype(np.float32)
init_pos = np.dstack(np.meshgrid(init_pos_x, init_pos_y)).reshape(-1, 2)
n_particles = init_pos.shape[0]
pbf = PBF(np.array([50, 50]), init_pos, interaction_distance, 8)

In [ ]:
n_burn_in = 5000

In [ ]:
traj = pbf.run(n_burn_in, 0);

In [ ]:
def make_animation(trajectory, stride, mode="scatter"):
    traj = trajectory.reshape((len(trajectory), -1, 2))
    traj = traj[::stride]
    
    backend_ =  mpl.get_backend()
    mpl.use("Agg")  # Prevent showing stuff

    fig, ax = plt.subplots()
    ax.set_xlim((-25, 25))
    ax.set_ylim((-25, 25))

    s = np.empty((n_particles,))
    s.fill(300)
    c = np.empty((n_particles,))
    c.fill(0.5)
    
    # needed for contourf
    grid = None
    gridx = None
    gridy = None
    contour_handle = []
    
    if mode == "scatter":
        the_plot = ax.scatter(traj[0, :, 0], traj[0, :, 1], s=s, c=c, vmin=0, vmax=1, 
                              cmap="jet", edgecolor="k")
    elif mode == "contourf":
        gridx = np.arange(-25, 25, box_width).astype(np.float32)
        gridy = np.arange(-25, 25, box_width).astype(np.float32)
        grid = np.meshgrid(gridx, gridy)
        the_plot = ax.contourf(grid[0], grid[1], traj[0].reshape((len(gridx), len(gridy))))
        contour_handle.append(the_plot)
    
    def update_scatter(i):
        X = traj[i]
        the_plot.set_offsets(X)
        return the_plot,
        
    def update_contourf(i):
        X = traj[i]
        for tp in contour_handle[0].collections:
            tp.remove()
        contour_handle[0] = ax.contourf(grid[0], grid[1], X.reshape((len(gridx), len(gridy))))
        return contour_handle[0].collections
    
    update = update_scatter if mode == "scatter" else update_contourf
    ani = animation.FuncAnimation(fig, update, interval=50, blit=True, repeat=False, 
                                  frames=len(traj))

    mpl.use(backend_) # Reset backend
    return ani

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
n_steps = 350
n_rounds = 40
traj_total = np.empty((n_rounds*n_steps, n_particles*2))
for i in tqdm(range(n_rounds)):
    traj = pbf.run(n_steps, .7 if i % 2 != 0 else -.7);
    traj_total[i*n_steps:(i+1)*n_steps] = traj

In [ ]:
ani = make_animation(traj_total, 30)
from IPython.display import HTML
HTML(ani.to_html5_video())

In [ ]:
from scipy.stats import gaussian_kde
box_width = .7
traj = traj_total.reshape(len(traj_total), -1, 2)

In [ ]:
import multiprocessing as mp

gridx = np.arange(-25, 25, box_width).astype(np.float32)
gridy = np.arange(-25, 25, box_width).astype(np.float32)
grid = np.meshgrid(gridx, gridy)
kde_input = np.dstack(grid).reshape(-1, 2)
traj_kde = np.empty((len(traj), len(kde_input)))

def worker(args):
    from scipy.stats import gaussian_kde
    t, traj = args[0], args[1]
    out = gaussian_kde(traj.T, bw_method=0.2).evaluate(kde_input.T)
    out /= out.sum()
    return t, out

with mp.Pool(processes=8) as pool:
    for result in tqdm(pool.imap_unordered(worker, zip(range(len(traj)), traj)), total=len(traj)):
        traj_kde[result[0]] = result[1]

In [ ]:
ani = make_animation(traj_kde, 30, "contourf")
from IPython.display import HTML
HTML(ani.to_html5_video())

In [ ]:
for t in range(len(traj_kde)):
    traj_kde[t] /= traj_kde[t].sum()

In [ ]:
est = sktime.decomposition.VAMP(lagtime=250, dim=.95)  # 10
est.fit(traj_kde, n_splits=10)
model = est.fetch_model()
print("dim:", model.output_dimension)
print("score:", model.score())
plt.plot(model.singular_values, '.')

In [ ]:
n_sing = 4

f, axes = plt.subplots(ncols=2, nrows=4, figsize=(12, 20))
for i in range(n_sing):
    ax = axes[i][0]
    im = model.singular_vectors_left[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("left eigenfunction {}".format(i))
    
    ax = axes[i][1]
    im = model.singular_vectors_right[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("right eigenfunction {}".format(i))

In [ ]:
est = sktime.decomposition.VAMP(lagtime=100, dim=.95)  # 10
est.fit(traj_kde, n_splits=10)
model = est.fetch_model()
print("dim:", model.output_dimension)
print("score:", model.score())
plt.plot(model.singular_values, '.')

In [ ]:
n_sing = 4

f, axes = plt.subplots(ncols=2, nrows=4, figsize=(12, 20))
for i in range(n_sing):
    ax = axes[i][0]
    im = model.singular_vectors_left[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("left eigenfunction {}".format(i))
    
    ax = axes[i][1]
    im = model.singular_vectors_right[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("right eigenfunction {}".format(i))

In [ ]:
est = sktime.decomposition.VAMP(lagtime=50, dim=.95)  # 10
est.fit(traj_kde, n_splits=10)
model = est.fetch_model()
print("dim:", model.output_dimension)
print("score:", model.score())
plt.plot(model.singular_values, '.')

In [ ]:
n_sing = 4

f, axes = plt.subplots(ncols=2, nrows=4, figsize=(12, 20))
for i in range(n_sing):
    ax = axes[i][0]
    im = model.singular_vectors_left[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("left eigenfunction {}".format(i))
    
    ax = axes[i][1]
    im = model.singular_vectors_right[:, i].reshape((len(gridx), len(gridy)))
    cb = ax.imshow(im, origin="lower", cmap="bwr")
    f.colorbar(cb, ax=ax)
    ax.set_title("right eigenfunction {}".format(i))